This is a notebook where we build dataframes to debug our models and check if they actually work.

In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import optuna
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import TPESampler

from config.config_types import AppConfig
from utils.logging_utils import ExperimentLogger
from training_routine.trainer import Trainer            
from pipeline.walkforward import WFCVGenerator
from pipeline.preprocessing import preprocess
from hyperparams_search.search_utils import sample_hparams_into_cfg
#from hyperparams_search.torch_estimator import TorchFoldEstimator
#from hyperparams_search.randomsearch import RandomSearch

from utils.gpu_test import gpu_test
from utils.paths import CONFIG_DIR
from utils.custom_formatter import setup_logger






from utils.logging_utils import ExperimentLogger
from training_routine.trainer import Trainer            
from pipeline.walkforward import WFCVGenerator
from utils.gpu_test import gpu_test
from utils.paths import CONFIG_DIR, SP500COPY_PATH, SP500_PATH
from utils.custom_formatter import setup_logger
import pandas as pd
from models import create_model 



In [3]:
cfg = AppConfig.from_dict(f"{CONFIG_DIR}/debug.yaml")

preprocess(path=SP500_PATH)


wf = WFCVGenerator(config=cfg.walkforward)

df = wf.df_master


17:11:11 - Preprocessing - DEBUG - percentage of nan returns 0.0034% (preprocessing.py:93)
17:11:11 - WFCVGenerator - DEBUG - WFConfig(train=753d, val=251d, test=251d, lags=20, step=251, max_folds=None) (walkforward.py:29)
17:11:11 - WFCVGenerator - DEBUG - Loading data via preprocess() (walkforward.py:44)
17:11:13 - Preprocessing - DEBUG - percentage of nan returns 0.0034% (preprocessing.py:93)
17:11:13 - WFCVGenerator - DEBUG - Loaded df: shape=(3155303, 3) dtypes={'t': dtype('int16'), 'ret': Float64Dtype(), 'permno': Int64Dtype()} head=
 t       ret  permno
 0 -0.012107   10078
 1 -0.062092   10078
 2  0.001742   10078 (walkforward.py:66)


`cfg.walkforward` was:
```
walkforward:
  max_folds: 
  step: 251
  ratio_train: 3
  ratio_val: 1
  ratio_test: 1
  lags: 20
  #purge_days: 20
  #embargo_days: 20
```


In [5]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# Build generator as you normally do
gen = WFCVGenerator(config=cfg.walkforward)  # must produce df_master with feature_*, 'y' (real)
df = gen.df_master.copy()

# 1) pick feature matrix (all feature_* columns) and fixed weights
feat_cols = [c for c in df.columns if c.startswith("feature_")]
d = len(feat_cols)
rng = np.random.default_rng(42)
w = rng.normal(size=d)
w /= np.linalg.norm(w) + 1e-9  # scale + no divide by 0

# 2) synth target on the WHOLE master df (no future info used)
x = df[feat_cols].to_numpy()
sig = x @ w
noise = rng.normal(scale=0.2 * sig.std(), size=len(sig))  # SNR controlled
y_linear = sig + noise

# (optional) nonlinear
y_nl = np.maximum(sig, 0.0) + 0.1*(sig**2) + rng.normal(scale=0.2*sig.std(), size=len(sig))

# 3) write synthetic targets into a copy for evaluation
df_lin = df.copy() 
df_lin["y"] = y_linear
df_nl  = df.copy()
df_nl["y"]  = y_nl



17:11:30 - WFCVGenerator - DEBUG - WFConfig(train=753d, val=251d, test=251d, lags=20, step=251, max_folds=None) (walkforward.py:29)
17:11:30 - WFCVGenerator - DEBUG - Loading data via preprocess() (walkforward.py:44)


17:11:31 - Preprocessing - DEBUG - percentage of nan returns 0.0034% (preprocessing.py:93)
17:11:32 - WFCVGenerator - DEBUG - Loaded df: shape=(3155303, 3) dtypes={'t': dtype('int16'), 'ret': Float64Dtype(), 'permno': Int64Dtype()} head=
 t       ret  permno
 0 -0.012107   10078
 1 -0.062092   10078
 2  0.001742   10078 (walkforward.py:66)


In [7]:
df_lin

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,y,window
0,-0.012107,-0.062092,0.001742,-0.053913,0.056985,0.094783,-0.011914,-0.043408,0.038656,0.040453,...,0.043708,-0.024549,-0.062916,0.044234,-0.048412,-0.021463,-0.024370,0.046628,-0.088906,"(0, 20)"
1,0.054099,-0.088360,-0.052815,-0.058824,0.076823,0.119710,-0.029158,-0.060067,-0.005325,0.016657,...,0.037199,0.007384,-0.092147,0.041523,-0.024363,-0.059024,-0.085645,0.054420,-0.090728,"(0, 20)"
2,-0.001606,-0.033780,0.010544,-0.033498,0.013068,0.007291,-0.025613,-0.032571,0.018901,0.041159,...,-0.009346,-0.021226,-0.024096,0.015432,-0.033435,-0.006289,-0.005063,-0.003817,-0.032094,"(0, 20)"
3,-0.049069,-0.030249,-0.001835,0.029412,-0.007143,0.010791,-0.003559,0.042857,0.054795,0.014610,...,0.000000,0.047541,-0.010955,0.072785,-0.008850,-0.065476,-0.070064,0.065069,-0.007639,"(0, 20)"
4,-0.017335,-0.017641,-0.013468,0.019340,0.053571,0.005297,-0.007376,0.009554,0.007361,0.000000,...,-0.025346,-0.029551,-0.069428,-0.003927,0.001314,-0.028871,-0.022973,0.062241,-0.062384,"(0, 20)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133510,0.006730,0.015812,0.006074,0.001258,0.025000,-0.009682,0.019678,-0.031193,-0.017540,-0.018873,...,-0.000789,-0.026995,0.033969,-0.023298,0.000268,0.013264,0.003438,-0.002635,-0.001520,"(6268, 6288)"
3133511,0.002841,-0.000315,0.030433,0.002750,0.007617,-0.021873,0.000927,0.009369,0.005712,-0.006592,...,-0.043639,0.015960,0.024671,-0.007923,0.014416,-0.005316,-0.012437,-0.009888,0.014950,"(6268, 6288)"
3133512,-0.003985,-0.003147,-0.022958,-0.014186,-0.004223,-0.012889,-0.031428,0.002509,0.010711,-0.036689,...,-0.044334,-0.010584,0.000897,-0.012671,0.002722,0.008533,-0.002243,-0.005974,-0.004144,"(6268, 6288)"
3133513,-0.018253,0.000991,-0.000283,-0.016552,-0.021865,-0.011716,0.006944,-0.002020,-0.013525,0.005654,...,0.010047,-0.031339,-0.011300,0.007776,0.006784,0.011008,0.003460,-0.006591,-0.013699,"(6268, 6288)"


In [18]:
type(df_lin["window"][0])

tuple

In [8]:
df_lin.to_parquet("./dfmaster_lin.parquet")

In [24]:

df_test["window"][0]

array([ 0, 20])

In [9]:
df_nl

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,y,window
0,-0.012107,-0.062092,0.001742,-0.053913,0.056985,0.094783,-0.011914,-0.043408,0.038656,0.040453,...,0.043708,-0.024549,-0.062916,0.044234,-0.048412,-0.021463,-0.024370,0.046628,-0.002186,"(0, 20)"
1,0.054099,-0.088360,-0.052815,-0.058824,0.076823,0.119710,-0.029158,-0.060067,-0.005325,0.016657,...,0.037199,0.007384,-0.092147,0.041523,-0.024363,-0.059024,-0.085645,0.054420,0.010569,"(0, 20)"
2,-0.001606,-0.033780,0.010544,-0.033498,0.013068,0.007291,-0.025613,-0.032571,0.018901,0.041159,...,-0.009346,-0.021226,-0.024096,0.015432,-0.033435,-0.006289,-0.005063,-0.003817,-0.003770,"(0, 20)"
3,-0.049069,-0.030249,-0.001835,0.029412,-0.007143,0.010791,-0.003559,0.042857,0.054795,0.014610,...,0.000000,0.047541,-0.010955,0.072785,-0.008850,-0.065476,-0.070064,0.065069,0.003833,"(0, 20)"
4,-0.017335,-0.017641,-0.013468,0.019340,0.053571,0.005297,-0.007376,0.009554,0.007361,0.000000,...,-0.025346,-0.029551,-0.069428,-0.003927,0.001314,-0.028871,-0.022973,0.062241,0.003294,"(0, 20)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133510,0.006730,0.015812,0.006074,0.001258,0.025000,-0.009682,0.019678,-0.031193,-0.017540,-0.018873,...,-0.000789,-0.026995,0.033969,-0.023298,0.000268,0.013264,0.003438,-0.002635,-0.005609,"(6268, 6288)"
3133511,0.002841,-0.000315,0.030433,0.002750,0.007617,-0.021873,0.000927,0.009369,0.005712,-0.006592,...,-0.043639,0.015960,0.024671,-0.007923,0.014416,-0.005316,-0.012437,-0.009888,0.019142,"(6268, 6288)"
3133512,-0.003985,-0.003147,-0.022958,-0.014186,-0.004223,-0.012889,-0.031428,0.002509,0.010711,-0.036689,...,-0.044334,-0.010584,0.000897,-0.012671,0.002722,0.008533,-0.002243,-0.005974,-0.001540,"(6268, 6288)"
3133513,-0.018253,0.000991,-0.000283,-0.016552,-0.021865,-0.011716,0.006944,-0.002020,-0.013525,0.005654,...,0.010047,-0.031339,-0.011300,0.007776,0.006784,0.011008,0.003460,-0.006591,-0.005044,"(6268, 6288)"


In [10]:
df_nl.to_parquet("./dfmaster_nl.parquet")